#### Setup inicial

In [ ]:
!pip install -r "./requirements.txt"

In [ ]:
import os 
from huggingface_hub import login

wkdir = "C:\\Users\\PLC03\\Documents\\Git\\test-diffusers"
os.chdir(wkdir)

image_ade20k = f"{wkdir}/images/mapa_ADE20K_FIX.png"
image_depth = f"{wkdir}/images/mapa_DEPTH_FIX.png"
image_real = f"{wkdir}/images/mapa_REALISTIC.png"
image_edge = f"{wkdir}/images/mapa_EDGE.png"

login()

In [ ]:
import models as M
from utils import preprocess_image

#### Prompt base

In [ ]:
prompt = """Ultra-realistic aerial view of a modern city, captured from a high-resolution satellite or drone. Buildings, residential areas, roads, and parks are detailed and proportional, with natural lighting and depth. Clear atmosphere with soft haze near the horizon."""
negative_prompt = "Distorted buildings, warped roads, AI artifacts, unnatural lighting, low detail, fisheye effect."
num_images = 1

prompts = [prompt]* num_images
negative_prompts = [negative_prompt]* num_images

In [ ]:
tensor_size = (768, 576)
segment_tensor = preprocess_image(image_ade20k, tensor_size)
depth_tensor = preprocess_image(image_depth, tensor_size)
tile_tensor = preprocess_image(image_real, tensor_size)
canny_tensor = preprocess_image(image_edge, tensor_size)

#### Realistic Vision V2 + ControlNet

In [ ]:
diffuser = M.PlaceDiffusionModel(
    base_diffusion_model=M.RealisticVisionV2,
    controlnet_images=(
        (segment_tensor, M.ControlModes.segment), 
        (depth_tensor, M.ControlModes.depth),
        (canny_tensor, M.ControlModes.canny)
    )
)

In [ ]:
pipe = diffuser.pipeline

#### Realistic Vision V3 + ControlNet

In [ ]:
diffuser = M.PlaceDiffusionModel(
    base_diffusion_model=M.RealisticVisionV3,
    controlnet_images=(
        (segment_tensor, M.ControlModes.segment),
        (depth_tensor, M.ControlModes.depth),
        (canny_tensor, M.ControlModes.canny)
    ),
    use_dpm_scheduler=False # Incompativel com o modelo
)

In [ ]:
pipe = diffuser.pipeline

In [ ]:
output = pipe(
    prompt=prompts,
    negative_prompt=negative_prompts,
    image=diffuser.control_ref_images,
    num_inference_steps=50,
    guidance_scale=7.5,
    controlnet_conditioning_scale=[0.7, 0.9, 1.0],
)

output.images[0]

#### LoRa (JZCGXL026) + ControlNet

In [ ]:
diffuser = M.PlaceDiffusionModel(
    lora_model=M.JZCGXL026(base_local_path="./safetensors"),
    controlnet_images=(
        (segment_tensor, M.ControlModes.segment), 
        (depth_tensor, M.ControlModes.depth),
        (canny_tensor, M.ControlModes.canny)
    )
)

In [ ]:
pipe = diffuser.pipeline

In [ ]:
output = pipe(
    prompt=prompts,
    negative_prompt=negative_prompts,
    image=diffuser.control_ref_images,
    num_inference_steps=50,
    guidance_scale=7.5,
    controlnet_conditioning_scale=[0.7, 0.9, 1.0],
)

output.images[0]